# RAG test

In [3]:
# 1. Základní nastavení a importy
# -------------------------------

import os
import torch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Složka, kde máte uložené vaše PDF soubory
SOURCE_DOCS_DIR = "/home/luk-hrd/Dokumenty/Starchip_data/" 

# Složka, kam se uloží vektorová databáze
PERSIST_DB_DIR = "./rag_db"

# Název modelu pro embedding (převod textu na vektory)
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"

# Parametry pro dělení textu na chunky (stejné jako ve vašem plánu)
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

In [4]:
# 1. Načtení dokumentů
# --------------------

if not os.path.exists(SOURCE_DOCS_DIR):
    print(f"Chyba: Složka '{SOURCE_DOCS_DIR}' neexistuje. Vytvořte ji a vložte do ní své PDF soubory.")
else:
    pdf_files = [f for f in os.listdir(SOURCE_DOCS_DIR) if f.endswith('.pdf')]
    
    if not pdf_files:
        print(f"Chyba: Ve složce '{SOURCE_DOCS_DIR}' nebyly nalezeny žádné PDF soubory.")
    else:
        print(f"Nalezeno {len(pdf_files)} PDF souborů. Načítám obsah...")
        
        documents = []
        for pdf_file in pdf_files:
            file_path = os.path.join(SOURCE_DOCS_DIR, pdf_file)
            try:
                loader = PyPDFLoader(file_path)
                documents.extend(loader.load())
                print(f" - Úspěšně načten soubor: {pdf_file}")
            except Exception as e:
                print(f" - Chyba při načítání souboru {pdf_file}: {e}")

        print(f"\nCelkem načteno {len(documents)} stránek z dokumentů.")
        # Pro kontrolu si můžeme vypsat kousek prvního dokumentu
        if documents:
            print("\nUkázka obsahu první stránky:")
            print(documents[0].page_content[:300] + "...")

Nalezeno 3 PDF souborů. Načítám obsah...
 - Úspěšně načten soubor: 546729149-Fundamentals-of-Quantum-Computing-2021-9783030636890-2021.pdf
 - Úspěšně načten soubor: 770370183-Learn-Quantum-Computing-Using-Qiskit.pdf
 - Úspěšně načten soubor: ModernQuantumMechanics-Sakurai.pdf

Celkem načteno 1409 stránek z dokumentů.

Ukázka obsahu první stránky:
Fundamentals 
of Quantum 
Computing
Venkateswaran Kasirajan
Theory and Practice...


In [5]:
# 2. Dělení textu na chunky
# -------------------------

if not documents:
    print("Chyba: Nejsou k dispozici žádné dokumenty k rozdělení. Spusťte nejprve předchozí buňku.")
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP
    )
    chunked_docs = text_splitter.split_documents(documents)
    
    print(f"Původní počet stránek: {len(documents)}")
    print(f"Nový počet chunků: {len(chunked_docs)}")
    
    # Pro kontrolu si můžeme vypsat první chunk
    if chunked_docs:
        print("\nUkázka prvního chunku:")
        print(chunked_docs[0])

Původní počet stránek: 1409
Nový počet chunků: 3414

Ukázka prvního chunku:
page_content='Fundamentals 
of Quantum 
Computing
Venkateswaran Kasirajan
Theory and Practice' metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'PyPDF', 'creationdate': '2021-06-16T11:45:45+05:30', 'moddate': '2021-06-18T11:22:43+05:30', 'source': '/home/luk-hrd/Dokumenty/Starchip_data/546729149-Fundamentals-of-Quantum-Computing-2021-9783030636890-2021.pdf', 'total_pages': 480, 'page': 0, 'page_label': 'C1'}


In [8]:
# 3. Vytvoření a uložení vektorové databáze
# ----------------------------------------

if not chunked_docs:
    print("Chyba: Nejsou k dispozici žádné chunky pro vytvoření databáze. Spusťte nejprve předchozí buňky.")
else:
    print("Inicializuji embedding model...")

    # Zvolíme zařízení (GPU, pokud je dostupné)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Pro výpočet embeddingů bude použito zařízení: {device}")
    
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': device}
    )

    print(f"\nVytvářím embeddingy pro {len(chunked_docs)} chunků a ukládám do databáze...")
    print(f"Cílová složka: '{PERSIST_DB_DIR}'")
    print("Tento proces může trvat delší dobu...")
    
    # Vytvoření databáze a její uložení na disk
    db = Chroma.from_documents(
        documents=chunked_docs,
        embedding=embeddings,
        persist_directory=PERSIST_DB_DIR
    )
    
    print("\n Vektorová databáze byla úspěšně vytvořena a uložena.")

Inicializuji embedding model...
Pro výpočet embeddingů bude použito zařízení: cuda

Vytvářím embeddingy pro 3414 chunků a ukládám do databáze...
Cílová složka: './rag_db'
Tento proces může trvat delší dobu...

 Vektorová databáze byla úspěšně vytvořena a uložena.


In [12]:
# 4. Testování a ověření databáze
# -------------------------------

print("Načítám existující vektorovou databázi...")

# Zvolíme zařízení (GPU, pokud je dostupné)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Používám zařízení: {device} pro načtení embedding modelu.")

# Načtení embedding modelu (stejného jako při vytváření)
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': device}
)

# Načtení databáze z disku
db = Chroma(
    persist_directory=PERSIST_DB_DIR, 
    embedding_function=embeddings
)

print("Databáze úspěšně načtena.")

# --- Samotný test ---
# Zadejte sem dotaz, který se týká obsahu vašich dokumentů
query = "What command in Qiskit is used to obtain a list of all available backends from the IBMQ provider?"

print(f"\nVyhledávám nejrelevantnější chunky pro dotaz: '{query}'")

# Provedeme vyhledávání. k=3 znamená, že chceme 3 nejrelevantnější výsledky.
# Vrací seznam dokumentů (chunků) a jejich skóre relevance
retrieved_docs = db.similarity_search_with_score(query, k=3)

print("\nNalezené chunky:")
print("-" * 50)

if not retrieved_docs:
    print("Nebyly nalezeny žádné relevantní chunky.")
else:
    for i, (doc, score) in enumerate(retrieved_docs):
        print(f"Výsledek č. {i+1} (Skóre: {score:.4f}):")
        print(doc.page_content)
        # Vypíšeme si i zdroj (název souboru a číslo stránky)
        print(f"\nZdroj: {doc.metadata.get('source', 'Neznámý')}, Strana: {doc.metadata.get('page', 'Neznámá')}")
        print("-" * 50)

Načítám existující vektorovou databázi...
Používám zařízení: cuda pro načtení embedding modelu.
Databáze úspěšně načtena.

Vyhledávám nejrelevantnější chunky pro dotaz: 'What command in Qiskit is used to obtain a list of all available backends from the IBMQ provider?'

Nalezené chunky:
--------------------------------------------------
Výsledek č. 1 (Skóre: 0.6708):
<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>
Now let's see what additional backends we have available.
[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='ma
 <IBMQBackend('ibmqx4') from IBMQ(hub='ibm-q', group='open', project='main')>, 
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>, 
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>]
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()
Here there is one si

In [ ]:
# 5. Inkrementální aktualizace databáze
# ------------------------------------
# Tento skript zkontroluje složku s dokumenty a zpracuje pouze ty soubory,

# Zkontrolujeme, zda databáze vůbec existuje
if not os.path.exists(PERSIST_DB_DIR):
    print(f"Chyba: Databáze ve složce '{PERSIST_DB_DIR}' neexistuje. Nejprve spusťte buňky pro její vytvoření.")
else:
    print("Spouštím proces aktualizace databáze...")
    
    # --- Krok 1: Načtení stávající databáze a zjištění zpracovaných souborů ---
    print("Načítám existující databázi pro kontrolu...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': device}
    )
    db = Chroma(
        persist_directory=PERSIST_DB_DIR, 
        embedding_function=embeddings
    )
    
    # Získáme metadata všech dokumentů v DB a z nich unikátní názvy zdrojových souborů
    # Používáme set pro efektivní operace
    existing_docs_metadata = db.get(include=["metadatas"])['metadatas']
    processed_files = {os.path.basename(meta['source']) for meta in existing_docs_metadata}
    print(f"V databázi je již zpracováno {len(processed_files)} souborů.")

    # --- Krok 2: Zjištění souborů na disku a porovnání ---
    all_disk_files = {f for f in os.listdir(SOURCE_DOCS_DIR) if f.endswith('.pdf')}
    
    # Najdeme soubory, které jsou na disku, ale ne v databázi
    new_files_to_process = all_disk_files - processed_files
    
    # --- Krok 3: Zpracování pouze nových souborů ---
    if not new_files_to_process:
        print("\n Databáze je aktuální. Nebyly nalezeny žádné nové soubory ke zpracování.")
    else:
        print(f"\nNalezeno {len(new_files_to_process)} nových souborů k zpracování:")
        for file in new_files_to_process:
            print(f" - {file}")
            
        new_documents = []
        for pdf_file in new_files_to_process:
            file_path = os.path.join(SOURCE_DOCS_DIR, pdf_file)
            try:
                loader = PyPDFLoader(file_path)
                new_documents.extend(loader.load())
            except Exception as e:
                print(f" - Chyba při načítání souboru {pdf_file}: {e}")
        
        print(f"\nNačteno {len(new_documents)} nových stránek. Dělím na chunky...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP
        )
        new_chunked_docs = text_splitter.split_documents(new_documents)
        print(f"Vytvořeno {len(new_chunked_docs)} nových chunků.")
        
        # --- Krok 4: Přidání nových chunků do existující databáze ---
        if new_chunked_docs:
            print("Přidávám nové chunky do existující databáze...")
            db.add_documents(documents=new_chunked_docs)
            # Metoda persist() není u Chroma explicitně potřeba, ukládá se průběžně
            print("\n Hotovo! Databáze byla úspěšně aktualizována o nové dokumenty.")
        else:
            print("\nNepodařilo se vytvořit žádné nové chunky ke zpracování.")

Spouštím proces aktualizace databáze...
Načítám existující databázi pro kontrolu...


/tmp/ipykernel_5610/2727123270.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/tmp/ipykernel_5610/2727123270.py:26: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


V databázi je již zpracováno 3 souborů.

 Databáze je aktuální. Nebyly nalezeny žádné nové soubory ke zpracování.
